In [3]:
import gmplot
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import peakutils 
import scipy
from __future__ import division

In [10]:
df_File = pd.read_csv('Jan021623.csv')          #Target Variable
time = df_File['Trip Time(Since journey start)(s)']
Accx = df_File['Acceleration Sensor(X axis)(g)']
Accy = df_File['Acceleration Sensor(Y axis)(g)']
Accz = df_File['Acceleration Sensor(Z axis)(g)']
Lat = df_File[' Latitude']
Longi = df_File[' Longitude']


In [ ]:
def Plot(x_direction,y_direction,z_direction,Time_parameter,x_label,y_label,plot_title):
  
    plt.figure()
    plt.plot(Time_parameter,x_direction,'b')
    plt.plot(Time_parameter,y_direction,'r')
    plt.plot(Time_parameter,z_direction,'c')
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.title(plot_title)
    
Plot(Accx,Accy,Accz,time,'Time','Acceleration','Acceleration in x y z directions')

## Function to take the average of every 3 values consequetively 

In [ ]:
def average(list):
    #list1 = [w.replace('Acceleration Sensor(Z axis)(g)', '0') for w in list] 
    list2 = [float(i) for i in list]
    res = 0 
    for ele in list2: 
        res += ele
        avg= res/3
    res = [ prod(list2[x : x + 3]) for x in range(0, len(list2), 3)]
    return (res)


## Function to pass the list through highpass filter

Cut off frequency(fc): 0.2 for sampling rate of 1 value/second (fraction of sampling rate)

Transition band (b): 0.16 for sampling rate of 1 value/second (fraction of sampling rate)

In [83]:
def highpassfilter(Accelerometer_values):
    fc = 0.2  
    b = 0.16  
    N = int(np.ceil((4 / b)))
    if not N % 2: N += 1  # Make sure that N is odd.
    n = np.arange(N)
 
    #Compute a low-pass filter.
    h = np.sinc(2 * fc * (n - (N - 1) / 2))
    w = np.blackman(N)
    h = h * w
    h = h / np.sum(h)
 
    #Create a high-pass filter from the low-pass filter through spectral inversion.
    h = -h
    h[(N - 1) // 2] += 1
    s = np.convolve(Accelerometer_values, h)
    np.set_printoptions(suppress=False)
    s1=s.astype(float) #amplifying the output by 10 times for further computation
    return s1

accy = highpassfilter(Accy)
accx = highpassfilter(Accx)

## Function to pass the list through a threshold

In [84]:
print(max(accy))
print(min(accy)) 
print(max(accx))
print(min(accx))

0.5232232001965982
-0.4332521479898295
0.4743616823522766
-0.8117229361741394


Assumptions: Any value less than -0.2 (Confirmed pothole)
               
Values (-0.2<x<-0.1) (Possible pothole)

In [90]:
def threshold(list):
    values = np.array(list)
    confirmed_potholes = values[values<-0.2]
    indexvalues_confirmed = np.nonzero(accy <-0.2)
    rest = np.array(list[list>-0.2])
    possible_potholes = rest[rest<-0.1]
    return confirmed_potholes,possible_potholes,indexvalues_confirmed
    
threshold(accy)

(array([-0.24487613, -0.3575369 , -0.43325215, -0.34919881]),
 array([-0.18257966, -0.1171724 , -0.18231727, -0.10082313, -0.11801906,
        -0.18542902, -0.18228244, -0.11702894, -0.10007617, -0.19352129,
        -0.12265666, -0.11830497, -0.14070854, -0.16148422, -0.18583374,
        -0.10045066, -0.17584293, -0.11395598, -0.10653603, -0.19073978]),
 (array([ 34,  38,  42, 421], dtype=int64),))

The number of confirmed potholes from the above function is 4, and the values are printed. The values of possible potholes are also printed. 

The last array indicates the index of the detected potholes from the list obtained from the high-pass filter function.

## Function to plot the coordinates on the map

In [ ]:
def PlotMap(latitude,Longitude):
    gmap = gmplot.GoogleMapPlotter(12.9716,77.5946,13)
    gmap.scatter(Latitude, Longitude,  
           'yellow',size = 40, marker = False) 
    gmap.draw("Potholes.html")